<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import biom
from biom.util import biom_open
import matplotlib.pyplot as plt
from q2_matchmaker._matching import _matchmaker
%matplotlib inline

In [2]:
!ls Averina2020/bioms

ogus.biom	     ogus_func-corrected.biom  species_func.biom
ogus-corrected.biom  species.biom	       virome.biom
ogus_func.biom	     species-corrected.biom


In [3]:
table1 = biom.load_table('Averina2020/bioms/virome.biom')
table2 = biom.load_table('Dan2020/bioms/virome.biom')
table3 = biom.load_table('PTHS/bioms/virome.biom')
table4 = biom.load_table('Wang2021/bioms/virome.biom')

In [4]:
md1 = pd.read_table('Averina2020/sample_metadata.txt')

# Dan2020
lookup = {'TD': 'Control', 'Autism': 'ASD'}
md2 = pd.read_table('Dan2020/sample_metadata.txt')
md2['Status'] = md2['Status'].apply(lambda x: lookup[x])

# PTHS TODO need to fix metadata

# Wang2020
lookup = {'TD': 'Control', 'ASD': 'ASD'}
md4 = pd.read_table('Wang2021/sample_metadata.txt', comment='#')
md4['Status'] = md4['Group'].apply(lambda x: lookup[x])

#md4['Age'] = md4['Sex']
lookup = {'M': 'male', 'F': 'female'}
md4['Sex'] = md4['Sex'].apply(lambda x: lookup[x])
md4['Match_IDs'] = _matchmaker(md4, 'Status', ['Age', 'Sex'], [False, True])
md4 = md4.dropna(subset=['Match_IDs'])
md4['Match_IDs'] = md4['Match_IDs'].apply(lambda x: f'Wang2020_{int(x)}')

In [5]:
md1 = md1.set_index('Run')
md2 = md2.set_index('SRA shotgun')
md4 = md4.set_index('sampleid')

md1.index.name = 'sampleid'
md2.index.name = 'sampleid'
md4.index.name = 'sampleid'

sample_md = pd.concat((md1, md2, md4), axis=0)

In [6]:
# merge all of the biom tables together
#table = table1.merge(table2).merge(table3).merge(table4)
table = table1.merge(table2).merge(table4)

In [7]:
def match_f(table, metadata):
    filter_f = lambda v, i, m: i in metadata.index
    t = table.filter(filter_f, axis='sample')
    md = metadata.loc[table.ids()]
    return t, md
table, sample_md = match_f(table, sample_md)

In [8]:
with biom_open('Combined/viral_table.biom', 'w') as f:
    table.to_hdf5(f, 'viruses')